In [1]:
import pandas as pd
from IPython.core.display import display, HTML
import torch
from torch.nn import DataParallel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
# from captum.attr import ShapleyValueSampling

/tmp/ipykernel_3677111/1278183268.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
attribution_file = "lime20_attri.pt" #change this file name to load a different attribution file
# attribution_file = "shap20_attri.pt"
attributions = torch.load(attribution_file)

In [3]:
attributions.keys()

dict_keys(['raw_text', 'all_input_ids', 'attributions', 'attribution_method', 'base_model_name'])

In [4]:
attributions['attribution_method']

'lime'

In [5]:
base_model_name = attributions['base_model_name']
tokenizer = AutoTokenizer.from_pretrained(base_model_name)  # Load tokenizer from Hugging Face's transformers

In [6]:
def z_score_normalize(tensor):
    mean = torch.mean(tensor)
    std = torch.std(tensor)
    normalized_tensor = (tensor - mean) / std
    return normalized_tensor

def generate_html(words, importance):
    html_content = "<html><body><p>"
    
    for word, imp in zip(words, importance):
        if imp < 0:
            color_intensity = int(255 * (1 + imp / min(importance)))  # Negative values will be red
            color = f"rgb(12, {color_intensity}, {color_intensity})"  # Red shade based on negative importance
        else:
            color_intensity = int(255 * (1 - imp / max(importance)))  # Positive values will be green
            color = f"rgb({color_intensity}, 255, {color_intensity})"  # Green shade based on positive importance
            
        html_content += f'<span style="background-color: {color}; padding: 2px; margin: 2px;">{word}</span> '
    
    html_content += "</p></body></html>"
    return html_content

In [7]:
for input_ids, attr in zip(attributions['all_input_ids'], attributions['attributions']):
    # print(input_ids.shape)
    # print(attr)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    normalized_attr = z_score_normalize(attr)
    print(normalized_attr)
    
    html_output = generate_html(tokens, normalized_attr)
    display(HTML(html_output))
    break

tensor([ -4.8142,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,
          0.0662,   0.0662,  -0.7662,   0.0662,   0.1571,   0.0662,   0.0662,
          0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,
          0.0662,   1.0232,   0.0662,   0.0662,  -1.2529,   0.0662,   0.0662,
          0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,
          0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,
          0.0662,   0.0662,   0.0662,   1.3561,   0.0662,   0.0662,   0.0662,
          0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,
          0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,
          0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,
          0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,  -1.3768,
         -0.1373,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,   0.0662,
         -1.0033,   0.0662,   0.0662,   0.0662,   0.0662,   0.06